# LDA에서 Author Topic Model

<img src="figs/atm_fig.png">

* LDA에서 저자정보(metadata)가 추가된 토픽모델.
* 문서별 단어분포를 반영하던 파라미터에, 저자별 문서분포 정보까지 추가된 형태.

<img src="figs/atm_fig2.png">

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

import pickle

In [2]:
with open("./cleaned_data.pk", "rb") as f:
    data = pickle.load(f)
    
data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

         Date   Speaker timetype   time contents
0  2021-06-14  김태환 형 17       오전   6:01      고민중
1  2021-06-14    이현직 16       오전   8:00      셤잘쳐
2  2021-06-14  김태환 형 17       오전   8:00      귀여워
3  2021-06-14  김태환 형 17       오전  10:07       시발
4  2021-06-14  김태환 형 17       오전  10:08   담배가 쓰다
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      980 non-null    object
 1   Speaker   980 non-null    object
 2   timetype  980 non-null    object
 3   time      980 non-null    object
 4   contents  980 non-null    object
dtypes: object(5)
memory usage: 38.4+ KB
None


### ATM 사용을 위한 데이터 처리

In [3]:
users = set(data["Speaker"])
users

{'김동현', '김락원 형 15', '김태환 형 17', '김표선 형 16', '이현직 16', '허정훈이 형 15'}

In [4]:
# 사용자별로 데이터 묶기
authors = data.groupby("Speaker")
pprint(authors.groups)
print(type(authors.groups))

{'김동현': Int64Index([102, 104, 135, 136, 148, 150, 152, 205, 206, 211, 212, 216, 239,
            401, 402, 405, 406, 427, 431, 468, 470, 475, 476, 477, 482, 487,
            488, 489, 492, 495, 498, 548, 584, 585, 586, 668, 669, 693, 701,
            934, 960, 964, 973, 974],
           dtype='int64'),
 '김락원 형 15': Int64Index([  9,  28,  30,  33,  36,  46,  47, 100, 223, 225, 237, 380, 385,
            389, 390, 398, 421, 424, 454, 465, 478, 505, 506, 536, 600, 603,
            677, 684, 708, 712, 776, 779, 780, 785, 790, 792, 815, 818, 821,
            825, 828, 830, 835, 840, 855, 858, 860, 890, 904, 918, 924, 927,
            931, 935, 937, 941, 944, 947, 954, 957],
           dtype='int64'),
 '김태환 형 17': Int64Index([  0,   2,   3,   4,   6,   7,   8,  18,  26,  27,
            ...
            963, 965, 966, 967, 968, 969, 970, 975, 976, 977],
           dtype='int64', length=317),
 '김표선 형 16': Int64Index([ 12,  13,  20,  21,  22,  37,  43,  44,  45,  49,
            ...
           

In [5]:
# 묶인 데이터를 Int64Index에서 list로 바꾸기

author2doc = {}

for user, index in authors.groups.items():
    author2doc[user] = list(index)
    
print(author2doc)

{'김동현': [102, 104, 135, 136, 148, 150, 152, 205, 206, 211, 212, 216, 239, 401, 402, 405, 406, 427, 431, 468, 470, 475, 476, 477, 482, 487, 488, 489, 492, 495, 498, 548, 584, 585, 586, 668, 669, 693, 701, 934, 960, 964, 973, 974], '김락원 형 15': [9, 28, 30, 33, 36, 46, 47, 100, 223, 225, 237, 380, 385, 389, 390, 398, 421, 424, 454, 465, 478, 505, 506, 536, 600, 603, 677, 684, 708, 712, 776, 779, 780, 785, 790, 792, 815, 818, 821, 825, 828, 830, 835, 840, 855, 858, 860, 890, 904, 918, 924, 927, 931, 935, 937, 941, 944, 947, 954, 957], '김태환 형 17': [0, 2, 3, 4, 6, 7, 8, 18, 26, 27, 29, 31, 32, 34, 35, 38, 40, 42, 48, 50, 51, 53, 57, 61, 62, 63, 65, 66, 67, 68, 69, 70, 81, 82, 85, 87, 88, 89, 90, 93, 97, 99, 101, 103, 105, 113, 114, 118, 121, 126, 128, 137, 144, 158, 160, 165, 180, 182, 188, 191, 194, 199, 226, 232, 233, 235, 238, 241, 242, 247, 248, 252, 253, 255, 260, 264, 266, 269, 270, 271, 273, 279, 280, 282, 284, 288, 294, 296, 298, 300, 302, 303, 308, 312, 315, 320, 323, 324, 326, 329, 

In [6]:
# gensim의 들어갈 데이터 만들기

tokenized_data = [msg.split() for msg in list(data["contents"])]
print(tokenized_data[:10])

[['고민중'], ['셤잘쳐'], ['귀여워'], ['시발'], ['담배가', '쓰다'], ['왜'], ['담배가', '더', '쓰다'], ['락원아'], ['동현이가', '저녁', '먹으러', '오래'], ['오키오키']]


### gensim을 이용한 Author Topic Model

In [7]:
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora

import os

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [8]:
# ATM에 사용할 Dictionary 만들기
if not os.path.exists("kakao(ATM)_dict"):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save("kakao(ATM)_dict")
else:
    dictionary = Dictionary.load("kakao(ATM)_dict")
    
# ATM에 사용할 corpus 만들기
if not os.path.exists("kakao(ATM)_corpus"):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize("kakao(ATM)_corpus", corpus)
else:
    corpus = bleicorpus.BleiCorpus("kakao(ATM)_corpus")

In [9]:
# ATM에 들어갈 데이터 확인하기
print("Number of authors : %d" % len(authors))
print("Number of unique tokens : %d" % len(dictionary))
print("Number of documents : %d" % len(corpus))


Number of authors : 6
Number of unique tokens : 1793
Number of documents : 980


In [10]:
# dictionary함수로 만든 사전에 있는 단어 보기
print(dictionary)
for idx in dictionary:
    print(dictionary[idx])

Dictionary(1793 unique tokens: ['고민중', '셤잘쳐', '귀여워', '시발', '담배가']...)
고민중
셤잘쳐
귀여워
시발
담배가
쓰다
왜
더
락원아
동현이가
먹으러
오래
저녁
오키오키
동현아
마른
어캐하냐
유지
체형
26년
동안
동현이처럼
생활하면
같은
동현이
될걸
체형이
동현이가아직
26년안했는데
생활을
혹시모르잖아
26년째되면
생활이지
정자생활도
나처럼
쟤도
찔지
음
보니깐
사륜안으로
안찐데
미래도봐지나
몇년
살았을
알고
정자가
줄
궁금한게
되냐
순환
어떻게
정자의
주기가
며칠
안갈걸
근데
동현이는
슈퍼
정자라
몇시에
저녁먹을거임
6시
5시45분
구체적인
매우
시각이
의심스러운데
지나치게
때문임
배드민턴
혹시
배고픔
아니
미안
아
감
당장
지금
도서관에서
오냐
뛰어라
뛰는중
그럼
나도
뜀
이거
풀어보셈
어케푸노
갖다대네
부분오픈을
정의는
정확한
호미오로
와
이게
풀이냐
똑똑하네
초딩들
위상도배우고
감사
답
뭐임
락교수
얼른
풀어
교환법칙
성립하냐
연산
저
몰라
같이
니
야
오늘
저녁먹었어야
표선아
했음
견제
락원이
씹오졌다
니가
안불러주는데
어케같이먹냐
섭섭하다
빤스런한건
아까
쳐불렀을때
내가
선약이
있었으니깐
실망이다
진짜
내일
점심
학식
선영이
있어서
깔깔깔
같았다
이현직
하지마
서태완이냐
쉣
홀리
무빙
잘못했네
살벌했음
공부
서로
쇼를
안했따
이러면서
하는데
내용
락원인
배우지도
않은
위상2
있고
다
동현인
족보
진작에
풀었고
벡해
나한테
몇문제
못풀었다했는데
편미방중간고사
100점만점에
88임
93
나
난
다풀었는데
싯팔
십점인데
한문제당
7점
감점이면
못풀어서
못푼거냐
기만
지리네
기만질
웃기네
존나
락원이도
만만치
않음
가는
거
락원이는
부터가
운동하러
기만질인
그리고
속보
하나
뭔데
거꾸로
나오느라
옷
입음
허겁지겁
뭐다
이말은
누군가
있었다
집에
여기까지
그건
그렇게
태환아
나오면
뒤집어서
보통
입지는
정상적으로
누가
않지
있다고
해서고
내밖에
없제
편
15
김락원
형
이제와서
중립이였지
항상
내리막길에
넣어놨찌

In [11]:
# 사람이 이해할 수 있는 형태로 코퍼스 사전 재구성 해보기 (term-frequency)
# for cp in corpus:
#     print(cp)
    
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus]

[[('고민중', 1.0)],
 [('셤잘쳐', 1.0)],
 [('귀여워', 1.0)],
 [('시발', 1.0)],
 [('담배가', 1.0), ('쓰다', 1.0)],
 [('왜', 1.0)],
 [('담배가', 1.0), ('쓰다', 1.0), ('더', 1.0)],
 [('락원아', 1.0)],
 [('동현이가', 1.0), ('먹으러', 1.0), ('오래', 1.0), ('저녁', 1.0)],
 [('오키오키', 1.0)],
 [('동현아', 1.0)],
 [('마른', 1.0), ('어캐하냐', 1.0), ('유지', 1.0), ('체형', 1.0)],
 [('26년', 1.0), ('동안', 1.0), ('동현이처럼', 1.0), ('생활하면', 1.0)],
 [('같은', 1.0), ('동현이', 1.0), ('될걸', 1.0), ('체형이', 1.0)],
 [('동현이가아직', 1.0)],
 [('26년안했는데', 1.0), ('생활을', 1.0)],
 [('혹시모르잖아', 1.0)],
 [('26년째되면', 1.0)],
 [('생활이지', 1.0), ('정자생활도', 1.0)],
 [('나처럼', 1.0), ('쟤도', 1.0), ('찔지', 1.0)],
 [('음', 1.0)],
 [('보니깐', 1.0), ('사륜안으로', 1.0)],
 [('안찐데', 1.0)],
 [('미래도봐지나', 1.0)],
 [('몇년', 1.0), ('살았을', 1.0), ('알고', 1.0), ('정자가', 1.0), ('줄', 1.0)],
 [('궁금한게', 1.0),
  ('되냐', 1.0),
  ('순환', 1.0),
  ('어떻게', 1.0),
  ('정자의', 1.0),
  ('주기가', 1.0)],
 [('며칠', 1.0), ('안갈걸', 1.0)],
 [('근데', 1.0), ('동현이는', 1.0), ('슈퍼', 1.0), ('정자라', 1.0)],
 [('몇시에', 1.0), ('저녁먹을거임', 1.0)],
 [('6시', 1.0)],
 

In [12]:
# Author Topic Model 실행
NUM_TOPICS = 4

if not os.path.exists("kakao(ATM)_model"):
    model = AuthorTopicModel(corpus=corpus, id2word=dictionary.id2token, num_topics=NUM_TOPICS, author2doc=author2doc, passes=100)
    model.save("kakao(ATM)_model")
else:
    model = AuthorTopicModel.load("kakao(ATM)_model")

In [13]:
# 학습 결과 확인하기
model.show_topic(1) # 1번째 토픽

[('난', 0.012260236931585794),
 ('아', 0.011676997276547715),
 ('락원이', 0.010431516849827774),
 ('동현이', 0.005928134762841963),
 ('오늘', 0.005294233510440971),
 ('다', 0.005287299506846207),
 ('내가', 0.005243276235309308),
 ('근데', 0.004646346629371917),
 ('나', 0.004085344852820157),
 ('아니', 0.004019273777094868)]

In [14]:
model.show_topic(1, topn=20) # top20

[('난', 0.012260236931585794),
 ('아', 0.011676997276547715),
 ('락원이', 0.010431516849827774),
 ('동현이', 0.005928134762841963),
 ('오늘', 0.005294233510440971),
 ('다', 0.005287299506846207),
 ('내가', 0.005243276235309308),
 ('근데', 0.004646346629371917),
 ('나', 0.004085344852820157),
 ('아니', 0.004019273777094868),
 ('진짜', 0.004006107460260246),
 ('더', 0.004005512804978402),
 ('내일', 0.0034142604828852276),
 ('그럼', 0.003364904612905689),
 ('이걸', 0.003364637952624334),
 ('니', 0.002730084928202692),
 ('좀', 0.002724732961971116),
 ('락', 0.002723984236604207),
 ('너가', 0.0027237221128403382),
 ('이거', 0.0027237196641392526)]